In [ ]:
import os
import tensorflow as tf
import numpy as np
import cPickle
import math
from configuration import *
os.environ["THEANO_FLAGS"] = "device=cuda0"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
config_hardware = tf.ConfigProto()
config_hardware.gpu_options.per_process_gpu_memory_fraction = 0.15
tf.set_random_seed(9)

In [ ]:
from caption_gen.CaptionWraper import *

In [ ]:
from skipthought import encoder_manager

In [ ]:
from storyteller.inference_utils import vocabulary
from storyteller.inference_utils import story_generator
from storyteller import inference_wrapper

In [ ]:
FLAGS = tf.flags.FLAGS

tf.flags.DEFINE_string("checkpoint_path", "../StylishParaGen/bard_v1/res_adv_4/model.ckpt-174168",
                       "Model checkpoint file or directory containing a "
                       "model checkpoint file.")
tf.flags.DEFINE_string("vocab_file", "/media/VSlab3/kuanchen_arxiv/vocab_60000_advent_allpad/vocab_shared.txt", "Text file containing the vocabulary.")
tf.flags.DEFINE_string("book_data_dir", "/media/VSlab3/kuanchen_arxiv/BookCorpus_passage/", "directory of book data")
tf.flags.DEFINE_string("book_category", "Adventure", "cateogry of book")


tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
vse_config = vse_config()

In [ ]:
wrapper = CapgenWrapper(vse_config)

In [ ]:
image_path = "/media/VSlab3/kuanchen_arxiv/NeuralStoryTeller/neural-storyteller/images/universiades_girl.jpeg"
top_k = 100
captions = wrapper.get_caption(image_loc=image_path,k=top_k)

In [ ]:
encoder = encoder_manager.EncoderManager(config=config_hardware)

In [ ]:
encoder.load_model(model_config=stv_config(),
                   vocabulary_file="/media/VSlab3/kuanchen_arxiv/skip_thoughts_uni_2017_02_02/vocab.txt",
                   embedding_matrix_file="/media/VSlab3/kuanchen_arxiv/skip_thoughts_uni_2017_02_02/embeddings.npy",
                   checkpoint_path="/media/VSlab3/kuanchen_arxiv/skip_thoughts_uni_2017_02_02/model.ckpt-501424")

In [ ]:
vocab = vocabulary.Vocabulary(vocab_file=FLAGS.vocab_file, start_word='<sos>', end_word='<eos>',unk_word='<unk>')

In [ ]:
storyteller_config = storyteller_config()
storyteller_config.vocab_size = len(vocab.vocab)

In [ ]:
g = tf.Graph()
with g.as_default():
    model = inference_wrapper.InferenceWrapper()
    restore_fn = model.build_graph_from_config(storyteller_config,
                                               FLAGS.checkpoint_path)
g.finalize()

In [ ]:
description = np.mean(encoder.encode(captions),axis=0)

In [ ]:
encoder.close()

In [ ]:
with open("./style_bias/bias_cap.pkl", 'r') as handle:
    bias_source = cPickle.load(handle)
with open("./style_bias/bias_advent_long_100.pkl", 'r') as handle:
    bias_target_long = cPickle.load(handle)
with open("./style_bias/bias_advent_short_100.pkl", 'r') as handle:
    bias_target_short = cPickle.load(handle)

In [ ]:
description_style = description - bias_source + bias_target_long

In [ ]:
sess = tf.Session(graph=g,config=config_hardware)

In [ ]:
restore_fn(sess)
generator = story_generator.StoryGenerator(model, vocab, max_caption_length=100, beam_size=10)



stories = generator.beam_search(sess, np.expand_dims(description_style,0))
#captions = generator.beam_search(sess, stv_embed)
for i, story in enumerate(stories):
    # Ignore begin and end words.
    sentence = [vocab.id_to_word(w) for w in story.sentence[1:-1]]
    sentence = " ".join(sentence)
    print("  %d) %s (p=%f)" % (i, sentence, math.exp(story.logprob)))